# Import some library

In [1]:
import sys
sys.path.insert(0,'../../')
import pandas as pd
import numpy as np
import os
from lib.utils import general as g
from lib.genhelper.config_class import vcf_zarr_config as vzconfig
from lib.config.config_class import page_config, legend_config
import json

# Get and convert data from paper

This content will get variant id each reagion from result of [paper](https://www.biorxiv.org/content/10.1101/821504v1) to file.

## From input and output

In [2]:
# default data and format file .gen at reasult
chrom = '22'
regions = np.arange(10)+1
region_path_format = '../../rnnimp/results/chr{0}_{1}.gen'
obsers_path_format = '../../rnnimp/results/chr{0}_{1}.legend.gz'

In [3]:
output_file_format = './region_info/rnnimp.chr{}.r{:04d}'+page_config.tail[page_config.variant]
with g.writing('./region_info/config.json') as cf:
    json.dump({'file_name_format': os.path.abspath(output_file_format)},cf)

In [9]:
## this use for convert iput and output file from paper
chrom_name = vzconfig.chrom
pos_name = vzconfig.position
ref_name = vzconfig.ref
alt_name = vzconfig.alt
id_name = 'id'
flag_name = vzconfig.flag
for region in regions:
    region_path = region_path_format.format(chrom,region)
    obsers_path = obsers_path_format.format(chrom,region)
    output_path = output_file_format.format(chrom,region)
    folder = os.path.dirname(output_path)
    os.makedirs(folder,exist_ok=True)
    unobsers = pd.read_csv(region_path,sep=' ',header=None).iloc[:,2:5].copy()
    unobsers.columns = [pos_name,ref_name,alt_name]
    unobsers[chrom_name]= np.full(len(unobsers),chrom)
    unobsers[flag_name]= np.full(len(unobsers),0)
    obsers = pd.read_csv(obsers_path,sep=' ').iloc[:,1:5].copy()
    obsers.columns = [pos_name,ref_name,alt_name,flag_name]
    obsers[chrom_name] = np.full(len(obsers),chrom)
    cols = [chrom_name,pos_name,ref_name,alt_name,flag_name]
    final_data = pd.concat([unobsers[cols],obsers[cols]],ignore_index=True)
    final_data.sort_values(by=cols[:-1],ignore_index=True,inplace=True)
    final_data.to_csv(output_path,index=False,sep=page_config.page_split_params)

## From true file

In [4]:
# default data and format file .gen at reasult
chrom = '22'
regions = np.arange(329)+1
region_path_format = '../../rnnimp/example_data/model_data/region_{}.legend.gz'

In [5]:
output_file_format = './region_info/rnnimp.chr{}.r{:04d}'+page_config.tail[page_config.variant]
with g.writing('./region_info/config.json') as cf:
    json.dump({'file_name_format': os.path.abspath(output_file_format)},cf)

In [6]:
chrom_name = vzconfig.chrom
pos_name = vzconfig.position
ref_name = vzconfig.ref
alt_name = vzconfig.alt
af_name = vzconfig.af
id_name = 'id'
flag_name = vzconfig.flag
for region in regions:
    region_path = region_path_format.format(region)
    output_path = output_file_format.format(chrom,region-1)
    folder = os.path.dirname(output_path)
    os.makedirs(folder,exist_ok=True)
    paper_region = pd.read_csv(region_path,sep=legend_config.legend_split_params)
    paper_region.rename(columns={'position':pos_name,'a0':ref_name,'a1':alt_name,'a1_freq':af_name,'array_marker_flag':flag_name},inplace=True)
    paper_region[chrom_name] = np.full(paper_region.shape[0],chrom)
    cols = [chrom_name,pos_name,ref_name,alt_name,af_name,flag_name]
    final_data = paper_region[cols].copy()
    final_data.to_csv(output_path,index=False,sep=page_config.page_split_params)